In [1]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
from torchvision import datasets, transforms
from PIL import Image
from torch.autograd import Variable
import torch, os
import os
from torch import nn

In [2]:
!pip install torchvision

In [3]:
class PathImageFolder(datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(PathImageFolder, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [4]:
def load_data(data_path):
    dataTransformer = transforms.Compose([transforms.CenterCrop([256, 256]), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    transformedData = PathImageFolder(root=data_path, transform=dataTransformer)
    dataset = torch.utils.data.DataLoader(transformedData, batch_size=12, shuffle=True,
                                                            num_workers=2)
    return dataset

In [5]:
class AlexNet(nn.Module):

    def __init__(self, num_classes):
        
        super(AlexNet, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
             nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 7 * 7, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x2 = x
        for l in range(len(self.features)):
            x2 = self.features[l].forward(x2)
            self.features[l].AN = x2
        x = self.features(x)
        self.oldSize = x.size(0)
        x = x.view(x.size(0), 256 * 7 * 7)
        x2 = x
        for l in range(len(self.classifier)):
            x2 = self.classifier[l].forward(x2)
            self.classifier[l].AN = x2
        x = self.classifier(x)
        return x

In [6]:
def load_model(model_path, num_classes):
    net = AlexNet(num_classes)
    device = 'cpu'
    weights = torch.load(model_path, map_location=torch.device(device))
    net.load_state_dict(weights)
    net.to(device)
    return net

In [7]:
def train_model(net, train_data):
    correct = 0
    for idx, (batch_data, batch_classes, batch_paths) in enumerate(train_data):
        if torch.cuda.is_available(): batch_data, batch_classes = batch_data.cuda(), batch_classes.cuda()
        batch_data, batch_classes = Variable(batch_data), Variable(batch_classes)
        scores = net.forward(batch_data)
        pred = scores.data.max(1)[1]
        correct += torch.sum(pred == batch_classes.data).float()
        print('Accuracy', str(100*(correct.item()/len(list(train_data.dataset)))) + "%", 'Processed:', idx+1, end = '\r')
    print('Accuracy', str(100*(correct.item()/len(list(train_data.dataset)))) + "%")

In [8]:
def test_model(net, test_data):
    net.eval()
    correct = 0
    for idx, (batch_data, batch_classes, batch_paths) in enumerate(test_data):
        if torch.cuda.is_available(): batch_data, batch_classes = batch_data.cuda(), batch_classes.cuda()
        batch_data, batch_classes = Variable(batch_data), Variable(batch_classes)
        scores = net.forward(batch_data)
        pred = scores.data.max(1)[1]
        correct += torch.sum(pred == batch_classes.data).float()

In [9]:
from torchvision import datasets, transforms

def load_data(data_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  
        transforms.ToTensor(),
    ])

    dataset = datasets.ImageFolder(root=data_path, transform=transform)
    return dataset

In [10]:
train_data = load_data('Downloads/TrainingSet')

print('Class:Closed')

test_data = load_data('Downloads/TestSet')


print('Class:Closed')



Class:Closed
Class:Closed
